In [ ]:
# !pip install geemap

# Machine Learning with Earth Engine - Supervised Classification

## Supervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/classification

The `Classifier` package handles supervised classification by traditional ML algorithms running in Earth Engine. These classifiers include CART, RandomForest, NaiveBayes and SVM. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

![](https://i.imgur.com/vROsEiq.png)

## Step-by-step tutorial

### Import libraries

In [4]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-eslamelnahas-jupyter')

### Create an interactive map

In [6]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define a function that scales and masks Landsat 8 surface reflectance images.
def prep_sr_l8(image):
  """Scales and masks Landsat 8 surface reflectance images."""
  # Develop masks for unwanted pixels (fill, cloud, cloud shadow).
  qa_mask = image.select('QA_PIXEL').bitwiseAnd(0b11111).eq(0)
  saturation_mask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  def _get_factor_img(factor_names):
    factor_list = image.toDictionary().select(factor_names).values()
    return ee.Image.constant(factor_list)

  scale_img = _get_factor_img([
      'REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'])
  offset_img = _get_factor_img([
      'REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'])
  scaled = image.select('SR_B.|ST_B10').multiply(scale_img).add(offset_img)

  # Replace original bands with scaled bands and apply masks.
  return image.addBands(scaled, None, True).updateMask(
      qa_mask).updateMask(saturation_mask)


# Make a cloud-free Landsat 8 surface reflectance composite.
l8_image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate('2021-03-01', '2021-07-01')
    .map(prep_sr_l8)
    .median())

# Use these bands for prediction.
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# This property stores the land cover labels as consecutive
# integers starting from zero.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = l8_image.select(bands).sampleRegions(
    collection=points, properties=[label], scale=30
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = l8_image.select(bands).classify(trained)

# Display the inputs and the results.
m = geemap.Map()
m.set_center(31.5, 31, 8)
m.add_layer(
    l8_image,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.25},
    'image',
)
m.add_layer(
    classified,
    {'min': 0, 'max': 2, 'palette': ['orange', 'green', 'blue']},
    'classification',
)
m

### Add data to the map

In [9]:
# image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')

In [11]:
point=ee.Geometry.Point([31.5,31.5])
first = ee.ImageCollection('COPERNICUS/S2_SR') \
.filterBounds(point) \
.filterDate('2023-01-01', '2023-12-31') \
.sort('CLOUDY_PIXEL_PERCENTAGE') \
.first()
Map.centerObject(first, 10)
Map.addLayer(first, {"bands": ['B4', 'B3', 'B2'], "min": 0, "max": 4000}, 'first')


### Check image properties

In [13]:
ee.Date(first.get("system:time_start")).format("YYYY-MM-dd").getInfo()

'2023-08-06'

In [14]:
first.get("CLOUD_COVERAGE_ASSESSMENT").getInfo()

0.001971

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([31.217651, 31.102407, 31.992188, 31.594986])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([31.6049195, 31.3486965]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [16]:
# region = Map.user_roi
region = ee.Geometry.Rectangle([31.217651, 31.102407, 31.992188, 31.594986])
# region = ee.Geometry.Point([31.6049195, 31.3486965]).buffer(10000)

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create label dataset for training


![](https://i.imgur.com/7QoRXxu.png)

In [18]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)

Map.set_center(31.5, 31, 8)
Map.add_layer(dataset, {}, 'Land Cover')


In [19]:
# Make the training dataset.
points = dataset.sample(
    **{
        "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [20]:
print(points.size().getInfo())

5000


In [21]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [31.519665420596173, 31.479896261552543]}, 'id': '0', 'properties': {'discrete_classification': 200}}


### Train the classifier

In [23]:
# تحميل صورة Landsat
# image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')

# استخدام هذه النطاقات للتنبؤ
bands = ["B1","B2", "B3", "B4", "B5", "B6", "B7", "B8","B8A", "B9", "B11", "B12", "AOT", "WVP", "SCL","TCI_R", "TCI_G", "TCI_B", "MSK_CLDPRB", "MSK_SNWPRB", "QA10", "QA20", "QA60"]
 
# هذا المتغير يمثل الفئة المستهدفة للتصنيف
label = "discrete_classification"

# تراكب النقاط على الصورة للحصول على بيانات التدريب
training = first.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# تدريب مصنف CART باستخدام المعاملات الافتراضية
trained = ee.Classifier.smileCart().train(training, label, bands)


In [24]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'AOT': 180, 'B1': 1297, 'B11': 1041, 'B12': 1029, 'B2': 1371, 'B3': 1358, 'B4': 1121, 'B5': 1100, 'B6': 1060, 'B7': 1063, 'B8': 1056, 'B8A': 1058, 'B9': 1039, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': 0, 'QA20': 0, 'QA60': 0, 'SCL': 6, 'TCI_B': 38, 'TCI_G': 37, 'TCI_R': 13, 'WVP': 1919, 'discrete_classification': 200}}


### Classify the image

In [26]:
# Classify the image with the same bands used for training.
result = first.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=812.0, center=[31, 31.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [29]:
class_values = dataset.get("discrete_classification_class_values").getInfo()
class_values

[0,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 111,
 112,
 113,
 114,
 115,
 116,
 121,
 122,
 123,
 124,
 125,
 126,
 200]

In [30]:
class_palette = dataset.get("discrete_classification_class_palette").getInfo()
class_palette

['282828',
 'ffbb22',
 'ffff4c',
 'f096ff',
 'fa0000',
 'b4b4b4',
 'f0f0f0',
 '0032c8',
 '0096a0',
 'fae6a0',
 '58481f',
 '009900',
 '70663e',
 '00cc00',
 '4e751f',
 '007800',
 '666000',
 '8db400',
 '8d7400',
 'a0dc00',
 '929900',
 '648c00',
 '000080']

In [32]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [33]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=812.0, center=[31, 31.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

### Visualize the result

In [35]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [44]:
Map.add_legend(builtin_legend="COPERNICUS/Landcover/100m/Proba-V/Global")
Map

Map(bottom=107497.0, center=[31, 31.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…

### Export the result

Export the result directly to your computer:

In [46]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "landcover.tif")

In [72]:
geemap.ee_export_image(landcover, filename=out_file, scale=50)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Lenovo\Downloads\landcover.tif


Export the result to Google Drive:

In [68]:
geemap.ee_export_image_to_drive(
    landcover, description="landcover", folder="export", scale=10
)